# Solution
## Valentin Kopylov IAD4

In [0]:
## Libraries
import time
start = time.time()
!pip install lightfm
!pip install pymorphy2
import lightfm
import pymorphy2
import scipy.sparse
import json
import string
from tqdm.auto import tqdm 
from tqdm.auto import tqdm 
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix, eye, hstack
import json
import re
import time

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"username","key":"api-key"}

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

import kaggle
!echo '{"username":"wallykop","key":"ecce6c10f066cebc6533cf89c41ac3b0"}' > ~/.kaggle/kaggle.json
!kaggle competitions download -c recsys-iad-challenge


## Preprocessing
morph = pymorphy2.MorphAnalyzer()
def lemmatize_words(text):
    tokens = [morph.normal_forms(i)[0] for i in text.split()]
    tokens = [token for token in tokens if token != " " and token.strip() not in PUNCT_TO_REMOVE]
    text = " ".join(tokens)
    return text

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&«»0123456789№…👍'
PUNCT_TO_REMOVE = string.punctuation + punct
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
end = time.time()
print(f'{(end - start)} s')

     |████████████████████████████████| 307kB 23.6MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707631 sha256=aa7deaf17389c97da15d3c256336269074e48e2581427765d1236029d583050d
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 7.1MB 10.7MB/s 
 96% 353M/368M [00:04<00:00, 99.6MB/s]
100% 368M/368M [00:04<00:00, 91.4MB/s]
 94% 221M/235M [00:04<00:00, 62.6MB/s]
100% 235M/235M [00:05<00:00, 49.0MB/s]
 46% 5.00M/10.9M [00:00<00:00, 26.8MB/s]
100% 10.9M/10.9M [00:00<00:00, 43.4MB/s]
36.57055711746216 s


In [0]:
## Loading data
start = time.time()
!unzip items.json.zip
!unzip train.json.zip
!unzip random_benchmark.csv.zip
end = time.time()
print(f'{(end - start)} s')

Archive:  items.json.zip
  inflating: items.json              
Archive:  train.json.zip
  inflating: train.json              
Archive:  random_benchmark.csv.zip
  inflating: random_benchmark.csv    
31.037407159805298 s


In [0]:
## Preprocessing items (a lot time, approx. 3 hours)
start = time.time()
items_list=[]
with tqdm(open('./items.json')) as inf:
    for line in inf:
        item=json.loads(line)
        if isinstance(item['image'], float):
            item['image']=[0 for _ in range(96)]
            
        item['image'] = np.array(item['image'])
        items_list.append(item)

items=pd.DataFrame(items_list).set_index('itemId')
items['itemId'] = items.index

# Choose first 1500 symbols to use furthermore (in order to save time while lemmatization)
items ['content'] = items ['content'].str[:1500]
items.content = items.content.apply(lambda text: lemmatize_words(remove_punctuation(remove_urls(text.lower()))))
items.title = items.title.apply(lambda text: lemmatize_words(remove_punctuation(remove_urls(text.lower()))))
end = time.time()
print(f'{(end - start)} s')


11192.523244380951 s


In [0]:
## Loading train data
start = time.time()
import zipfile
train_zip = zipfile.ZipFile('/content/train.json.zip')
train_file = train_zip.filelist[0]

f_in = train_zip.open(train_file)

train = []
for line in tqdm(f_in):
    j = json.loads(line)
    user_items = []
    user_ratings = []
    for item, rating in j["trainRatings"].items():
        user_items.append(int(item))
        user_ratings.append(int(rating))
    train.append({"userId": j["userId"], 
                     "userItems": np.array(user_items), 
                     "userRatings": np.array(user_ratings)})
train = pd.DataFrame(train)

## Generating features of users
train['sum_clicks'] = [sum(i) for i in train.userRatings]
train['no_items'] = [len(i) for i in train.userItems]
train['cr'] = train['sum_clicks']/train['no_items'] 


user_features=['sum_clicks', 'no_items', 'cr']
features = train[user_features].apply(
        lambda x: ','.join(x.map(str)), axis=1)
features = features.str.split(',')
features1 = list(zip(train['userId'], features))
features = [(i, list(map(float, v))) for i, v in features1]
features1 = [v for i, v in features]
output_set = set()
for i in range(1, len(features1)):
  output_set.update(set(features1[i]).difference(output_set))

output_set.add(0.10329089598846986)


from lightfm.data import Dataset
dataset = Dataset()
dataset.fit(
    sorted(train.userId), 
    sorted(items.itemId),
   user_features=output_set)


user_features = dataset.build_user_features(
    features
    )
end = time.time()
print(f'{(end - start)} s')


91.14781475067139 s


In [0]:
start = time.time()
num_users = 42977
num_items = len(items)
data = []
row = []
col = []
end = time.time()
print(f'{(end - start)} s')

0.0004203319549560547 s


In [0]:
start = time.time()
with tqdm(open('./train.json')) as inf:
    for i, line in enumerate(inf):
        item=json.loads(line)
        for it, rate in item['trainRatings'].items():
          data.append((-1)**(int(rate) + 1))
          row.append(i)
          col.append(int(it))
end = time.time()
print(f'{(end - start)} s')


74.14782810211182 s


In [0]:
# Constructing interactions matrix of users and items
start = time.time()
interactions = coo_matrix((data, (row, col)))
end = time.time()
print(f'{(end - start)} s')

27.506031036376953 s


In [0]:
## Generating items' features
start = time.time()
from sklearn.feature_extraction.text import TfidfVectorizer

vect_content = TfidfVectorizer(lowercase=False)
tfidf_content = vect_content.fit_transform(items.content.apply(lambda x: np.str_(x)))

vect_title = TfidfVectorizer(lowercase=False)
tfidf_title = vect_content.fit_transform(items.title.apply(lambda x: np.str_(x)))

identity_items = eye(num_items)
item_features = hstack([identity_items, tfidf_content, tfidf_title], format="csr")

identity_users = eye(num_users)
user_features = hstack([identity_users, user_features], format='csr')

end = time.time()
print(f'{(end - start)} s')

64.00904321670532 s


In [0]:
del data, row, col

In [0]:
del items

In [0]:
## Model fitting (long time, approx. 8.3 hours)
start = time.time()
model = lightfm.LightFM(no_components=64, loss="logistic", random_state=346)
model.fit(interactions, 
          epochs=5, 
          item_features=item_features, 
          user_features=user_features, 
          verbose=True,
          num_threads=8)
end = time.time()
print(f'{(end - start)} s')

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
29787.636817216873 s


In [0]:
## Downloading predictions file
start = time.time()
predictions = pd.read_csv('random_benchmark.csv')
end = time.time()
print(f'{(end - start)} s')

0.3926985263824463 s


In [0]:
## Predicting values and arranging them
start = time.time()
predictions["rate"] = model.predict(
    predictions.userId.values,
    predictions.itemId.values,
    item_features=item_features,
    user_features=user_features
)

predictions.sort_values(["userId", "rate"], ascending=[True, False], inplace=True)
predictions.drop("rate", "columns", inplace=True)
predictions.to_csv("submission.csv", index=False)
end = time.time()
print(f'{(end - start)} s')

36.7498733997345 s


In [0]:
!kaggle competitions submit -c recsys-iad-challenge -f submission.csv -m "Final prediction"

100% 35.7M/35.7M [00:03<00:00, 10.6MB/s]
Successfully submitted to Articles RecSysWarning: Looks like you're using an outdated API Version, please consider updating (server 1.5.6 / client 1.5.4)
100% 35.7M/35.7M [00:03<00:00, 10.6MB/s]
Successfully submitted to Articles RecSysWarning: Looks like you're using an outdated API Version, please consider updating (server 1.5.6 / client 1.5.4)
100% 35.7M/35.7M [00:03<00:00, 10.6MB/s]
Successfully submitted to Articles RecSys